<h1>Φόρτωση βιβλιοθηκών</h1>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.model_selection import StratifiedKFold

import keras
from keras.models import Sequential # type: ignore
from keras.layers import Dense, Input # type: ignore
from keras.metrics import MeanSquaredError # type: ignore

import matplotlib.pyplot as plt


<h1>Προεπεξεργασία Δεδομένων</h1>

In [ ]:
# Unused columns
drop_cols = ['PatientID', 'DoctorInCharge']

categorical_cols = ['Gender', 'Ethnicity', 'EducationLevel']

numerical_cols = [
    'Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 
    'SystolicBP', 'DiastolicBP', 'CholesterolTotal',
    'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides',
    'MMSE', 'FunctionalAssessment', 'ADL'
]

# Preprocessing pipelines
standard_scaler = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers
preprocessor = ColumnTransformer(transformers=[
    ('num', standard_scaler, numerical_cols),
    ('cat', categorical_transformer, categorical_cols),
], remainder='passthrough')  # leaves binary columns as they are

# Load dataset
df = pd.read_csv("alzheimers_disease_data.csv")  # Load your real dataset
df = df.drop(columns=drop_cols)

# Separate input and output
X = df.drop(columns=['Diagnosis'])  # Adjust target if needed
Y = df['Diagnosis']

# Fit-transform your data
X = preprocessor.fit_transform(X)
input_processed = pd.DataFrame(X, columns=preprocessor.get_feature_names_out())

<h1>Παράμετροι Εκπαίδευσης</h1>

In [ ]:
I = X.shape[1]  # Number of columns
H = [
    I//2, 
    2*I//3, 
    I,
    2*I
          ]  # Hidden layer neurons

h = [0.001, 0.001, 0.05, 0.1]  # Learning rates
m = [0.2, 0.6, 0.6, 0.6] # Momentum

r = [0.0001, 0.001, 0.01]

# Split the data into a balanced 5-Fold
kfold = StratifiedKFold(n_splits=5, shuffle=True)

hidden_layer_activation = 'relu'  # Activation function for hidden layers
output_activation = 'sigmoid'  # For binary classification

epochs = 100
batch_size = 50

# Early stopping to prevent overfitting
early_stopping = keras.callbacks.EarlyStopping(
    monitor="loss",
    patience=5,
    restore_best_weights=True,
)

<h1>Εκπαίδευση και Προβολή Αποτελεσμάτων</h1>
<h2>Αλλαγή στον αριθμό των νευρώνων</h2>

In [ ]:
for i in range(len(H)):
    print("Number of neurons in the hidden layer: ", H[i])

    # Αποθήκευση τιμών για το διάγραμμα
    avg_loss_per_epoch = np.zeros(epochs)
    avg_mse_per_epoch = np.zeros(epochs)
    avg_acc_per_epoch = np.zeros(epochs)
    lossList = []
    mseList = []
    accuracyList = []

    for j, (train, test) in enumerate(kfold.split(X, Y)):
        # Δημιουργία μοντέλου
        model = Sequential()
        model.add(Input(shape=(I,)))
        model.add(Dense(H[i], activation=hidden_layer_activation))
        model.add(Dense(1, activation=output_activation))

        # Compile model
        optimizer = keras.optimizers.SGD(learning_rate=h[0], momentum=m[0])
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['mse', 'accuracy'])

        # Εκπαίδευση μοντέλου
        history = model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[early_stopping])

        # Αποθήκευση των μέσων τιμών για κάθε εποχή
        avg_loss_per_epoch += np.array(history.history['loss']) / kfold.get_n_splits()
        avg_mse_per_epoch += np.array(history.history['mse']) / kfold.get_n_splits()
        avg_acc_per_epoch += np.array(history.history['accuracy']) / kfold.get_n_splits()

        # Αξιολόγηση μοντέλου
        scores = model.evaluate(X[test], Y[test], verbose=0)
        lossList.append(scores[0])
        mseList.append(scores[1])
        accuracyList.append(scores[2])
        print(f"Fold {j}: Loss={scores[0]:.4f}, MSE={scores[1]:.4f}, Accuracy={scores[2]:.4f}")

    print(f"Average Loss: {np.mean(lossList):.4f}")
    print(f"Average MSE: {np.mean(mseList):.4f}")
    print(f"Average Accuracy: {np.mean(accuracyList):.4f}\n")
    
    # Σχεδίαση των γραφημάτων σύγκλισης
    plt.figure(figsize=(15, 5))

    # Loss Plot
    plt.subplot(1, 3, 1)
    plt.plot(range(1, epochs + 1), avg_loss_per_epoch, linestyle='-', color='r')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(f"Loss per Epoch (Hidden Neurons: {H[i]})")
    plt.grid()
    
    # MSE Plot
    plt.subplot(1, 3, 2)
    plt.plot(range(1, epochs + 1), avg_mse_per_epoch, linestyle='-', color='g')
    plt.xlabel("Epochs")
    plt.ylabel("MSE")
    plt.title(f"MSE per Epoch (Hidden Neurons: {H[i]})")
    plt.grid()

    # Accuracy Plot
    plt.subplot(1, 3, 3)
    plt.plot(range(1, epochs + 1), avg_acc_per_epoch, linestyle='-', color='b')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy per Epoch (Hidden Neurons: {H[i]})")
    plt.grid()

    plt.show()

<h2>Αλλαγή στις παραμέτρους εκμάθησης <b>h</b> και <b>m</b></h2>

In [ ]:
for i in range(len(h)):
    print(f"Learning Rate: {h[i]}")
    print(f"Momentum: {m[i]}")

    # Αποθήκευση τιμών για το διάγραμμα
    avg_loss_per_epoch = np.zeros(epochs)
    avg_mse_per_epoch = np.zeros(epochs)
    avg_acc_per_epoch = np.zeros(epochs)
    lossList = []
    mseList = []
    accuracyList = []

    for j, (train, test) in enumerate(kfold.split(X, Y)):
        # Δημιουργία μοντέλου
        model = Sequential()
        model.add(Input(shape=(I,)))
        model.add(Dense(H[3], activation=hidden_layer_activation))
        model.add(Dense(1, activation=output_activation))

        # Compile model
        optimizer = keras.optimizers.SGD(learning_rate=h[i], momentum=m[i])
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['mse', 'accuracy'])

        # Εκπαίδευση μοντέλου και αποθήκευση ιστορικού
        history = model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[early_stopping])

        # Αποθήκευση της απώλειας και της ακρίβειας ανά εποχή
        avg_loss_per_epoch += np.array(history.history['loss']) / kfold.get_n_splits()
        avg_mse_per_epoch += np.array(history.history['mse']) / kfold.get_n_splits()
        avg_acc_per_epoch += np.array(history.history['accuracy']) / kfold.get_n_splits()

        # Αξιολόγηση μοντέλου
        scores = model.evaluate(X[test], Y[test], verbose=0)
        lossList.append(scores[0])
        mseList.append(scores[1])
        accuracyList.append(scores[2])
        print(f"Fold {j}: Loss={scores[0]:.4f}, MSE={scores[1]:.4f}, Accuracy={scores[2]:.4f}")

    print(f"Average Loss: {np.mean(lossList):.4f}")
    print(f"Average MSE: {np.mean(mseList):.4f}")
    print(f"Average Accuracy: {np.mean(accuracyList):.4f}")
    
    # Σχεδίαση των γραφημάτων σύγκλισης
    plt.figure(figsize=(15, 5))

    # Loss Plot
    plt.subplot(1, 3, 1)
    plt.plot(range(1, epochs + 1), avg_loss_per_epoch, marker='o', linestyle='-', color='r')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(f"Loss per Epoch \n(Learning Rate: {h[i]}, Momentum: {m[i]})")
    plt.grid()
    
    # MSE Plot
    plt.subplot(1, 3, 2)
    plt.plot(range(1, epochs + 1), avg_mse_per_epoch, marker='o', linestyle='-', color='g')
    plt.xlabel("Epochs")
    plt.ylabel("MSE")
    plt.title(f"MSE per Epoch \n(Learning Rate: {h[i]}, Momentum: {m[i]})")
    plt.grid()

    # Accuracy Plot
    plt.subplot(1, 3, 3)
    plt.plot(range(1, epochs + 1), avg_acc_per_epoch, marker='o', linestyle='-', color='b')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy per Epoch \n(Learning Rate: {h[i]}, Momentum: {m[i]})")
    plt.grid()

    plt.show()

<h2>Ομαλοποίηση</h2>

In [ ]:
for i in range(len(r)):
    print(f"r: {r[i]}")

    # Αποθήκευση τιμών για το διάγραμμα
    avg_loss_per_epoch = np.zeros(epochs)
    avg_mse_per_epoch = np.zeros(epochs)
    avg_acc_per_epoch = np.zeros(epochs)
    lossList = []
    mseList = []
    accuracyList = []

    for j, (train, test) in enumerate(kfold.split(X, Y)):
        # Δημιουργία μοντέλου
        model = Sequential()
        model.add(Input(shape=(I,)))
        model.add(Dense(H[3], activation=hidden_layer_activation, kernel_regularizer=keras.regularizers.l2(r[i])))
        model.add(Dense(1, activation=output_activation, kernel_regularizer=keras.regularizers.l2(r[i])))

        # Compile model
        optimizer = keras.optimizers.SGD(learning_rate=h[1], momentum=m[1])
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['mse', 'accuracy'])

        # Εκπαίδευση μοντέλου και αποθήκευση ιστορικού
        history = model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[early_stopping])

        # Αποθήκευση της απώλειας και της ακρίβειας ανά εποχή
        avg_loss_per_epoch += np.array(history.history['loss']) / kfold.get_n_splits()
        avg_mse_per_epoch += np.array(history.history['mse']) / kfold.get_n_splits()
        avg_acc_per_epoch += np.array(history.history['accuracy']) / kfold.get_n_splits()

        # Αξιολόγηση μοντέλου
        scores = model.evaluate(X[test], Y[test], verbose=0)
        lossList.append(scores[0])
        mseList.append(scores[1])
        accuracyList.append(scores[2])
        print(f"Fold {j}: Loss={scores[0]:.4f}, MSE={scores[1]:.4f}, Accuracy={scores[2]:.4f}")

    print(f"Average Loss: {np.mean(lossList):.4f}")
    print(f"Average MSE: {np.mean(mseList):.4f}")
    print(f"Average Accuracy: {np.mean(accuracyList):.4f}")
    
    # Σχεδίαση των γραφημάτων σύγκλισης
    plt.figure(figsize=(15, 5))

    # Loss Plot
    plt.subplot(1, 3, 1)
    plt.plot(range(1, epochs + 1), avg_loss_per_epoch, marker='o', linestyle='-', color='r')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(f"Loss per Epoch \n(Learning Rate: {h[i]}, Momentum: {m[i]})")
    plt.grid()
    
    # MSE Plot
    plt.subplot(1, 3, 2)
    plt.plot(range(1, epochs + 1), avg_mse_per_epoch, marker='o', linestyle='-', color='g')
    plt.xlabel("Epochs")
    plt.ylabel("MSE")
    plt.title(f"MSE per Epoch \n(Learning Rate: {h[i]}, Momentum: {m[i]})")
    plt.grid()

    # Accuracy Plot
    plt.subplot(1, 3, 3)
    plt.plot(range(1, epochs + 1), avg_acc_per_epoch, marker='o', linestyle='-', color='b')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy per Epoch \n(Learning Rate: {h[i]}, Momentum: {m[i]})")
    plt.grid()

    plt.show()